In [1]:
using Test

In [2]:
f = readlines("day14.input");

In [3]:
function parse_int64(num)
    parse(Int64, num)
end

parse_int64 (generic function with 1 method)

In [4]:
function parse_input(input)
    min_x, max_x = 10000, 0
    min_y, max_y = 10000, 0

    rocks = Vector{Vector{Tuple{Int64, Int64}}}()
    for line in input
        rock = Vector{Tuple{Int64, Int64}}()
        for coord in split(line, "->")
            x, y = map(parse_int64, split(coord, ","))
            push!(rock, (x, y))
            max_x = max(x, max_x)
            max_y = max(y, max_y)
            min_y = min(y, min_y)
            min_x = min(x, min_x)

        end
        push!(rocks, rock)
    end

    rocks, (min_x, max_x), (min_y, max_y)
end

@test parse_input(String.(split("498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9", "\n"))) == ([[(498, 4), (498, 6), (496, 6)], [(503, 4), (502, 4), (502, 9), (494, 9)]], (494, 503), (4, 9))


Test Passed

In [5]:
function travel(p₀, p₁)
    diff = p₁ .- p₀
    step = sum(diff) > 0 ? 1 : -1
    if diff[1] != 0
        return [(x, p₀[2]) for x=p₀[1]:step:p₁[1]]
    end
    return [(p₀[1], y) for y=p₀[2]:step:p₁[2]]
end

travel (generic function with 1 method)

In [6]:
function build_world(rocks)
    world = Dict{Tuple{Int64, Int64}, Int64}()

    for rock in rocks
        for i=1:length(rock)-1
            from, to = rock[i], rock[i+1]
            for p in travel(from, to)
                world[p] = 1
            end
        end
    end

    world
end

build_world (generic function with 1 method)

In [7]:
function drop_sand(world, sand, floor)
    x, y = sand
    while y <= floor
        if !((x, y+1) in keys(world))
            y += 1
            continue
        elseif !((x-1, y+1) in keys(world))
            y += 1
            x -= 1
            continue
        elseif !((x+1, y+1) in keys(world))
            y += 1
            x += 1
            continue
        end

        world[(x, y)] = 2
        return true
    end

    return false
end

@test drop_sand(Dict(), (500, 0), 10) == false

Test Passed

In [8]:
function solve_part_1(input)
    rocks, x_range, y_range = parse_input(input)
    floor = y_range[2]
    world = build_world(rocks)
    dropped = 0
    while true
        if !drop_sand(world, (500, 0), floor)
            return dropped
        end
        dropped += 1
    end
end

solve_part_1 (generic function with 1 method)

In [9]:
function solve_part_2(input)
    rocks, x_range, y_range = parse_input(input)
    floor = y_range[2]+2
    world = build_world(rocks)
    for i=500-floor:500+floor
        world[i, floor] = 1
    end
    dropped = 0
    while true
        if !drop_sand(world, (500, 0), floor+1)
            return dropped
        end
        dropped += 1
        if (500, 0) in keys(world)
            return dropped
        end
    end
end

solve_part_2 (generic function with 1 method)

In [10]:
@test solve_part_1(String.(split("498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9", "\n"))) == 24


Test Passed

In [11]:
@test solve_part_2(String.(split("498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9", "\n"))) == 93

Test Passed

In [12]:
(solve_part_1(f), solve_part_2(f))

(715, 25248)